# 2. Functions in Haskell
**7 October 2021**

> **Recap from last session**: prefix and infix functions, left and right associativity of operators, conditional expressions, finding and forcing types, tuples, lists, `let... in...`, arithmetic sequences, list comprehensions, pattern matching


_Remember to complete the Haskell Sequences assignment on Friday (tomorrow) on CATe!_

## Defining functions

A function is a rule for associating each element of a source type A with a unique member of a target type B.
* `f :: A -> B`
* `f` will take an argument of type `A` and return a result of type `B`
* May take several arguments: types are listed in sequence
    * E.g. inputs A, B and C are defined as `g :: A -> B -> C -> D`, gives output D
    * Giving 'multiple' outputs: wrap into one tuple

A rule has a left-hand side (arguments) and a right-hand side (expression).

In [4]:
-- Define the I/O types
successor :: Int -> Int
-- Name the function and its input var
successor x 
-- Define the output expression (indent)
    = x + 1

More examples of functions
* `magnitude` generates the magnitude of a given vector
* `isUpper` checks if a character is an upper case letter
* `even` checks if a number is even

In [7]:
magnitude :: (Float, Float) -> Float
magnitude (x, y)
    = sqrt (x^2 + y^2)

isUpper :: Char -> Bool
isUpper ch
    = ch >= 'A' && ch <= 'Z'

isEven :: Int -> Bool
isEven x
    = x `mod` 2 == 0

Line 7: Use isAsciiUpper
Found:
ch >= 'A' && ch <= 'Z'
Why not:
isAsciiUpper chLine 11: Use even
Found:
x `mod` 2 == 0
Why not:
even x

* There may be constraints on what the function can take as arguments
    * Ideally try to prevent invalid arguments or state a precondition
* Use comments to specify preconditions in annotations
    * But use comments sparingly and wisely!

Apply functions by the juxtaposition `f a`.

In [9]:
successor 5
magnitude (13, 7)
isUpper 'g'
isEven 16

-- Composite functions
isEven (successor 19)

6

14.764823

False

True

True

## Guarded Rules

Rules (functions) can contain guards which generalise conditionals.
* `| predicate = true output`
* `| otherwise = false output`

In [11]:
difference :: Float -> Float -> Float
difference x y
    | x >= y    = x -y
    | otherwise = y - x
    
signum :: Int -> Int
signum n
    | n < 0     = -1
    | n == 0    = 0
    | otherwise = 1

Guards are evaluated from top to bottom.
* If guard condition is true: evaluate right expression
* If guard condition is false: proceed to next guard
* Run out of guards: move to the next rule in the function if it exists
* Run out of rules: error! 
    * Function is partial (not defined for entire domain

Definition of `otherwise`: default return value, same as `True`

## Local definitions

* `let` expressions introduce definitions local to an expression
* `where` clauses introduce definitions local to a rule
    * Set of definitions must be on the right hand side
    * This can only exist in a function

In [1]:
turns :: Float -> Float -> Float -> Float
turns start end r
    = totalDistance / distancePerTurn
      where
        totalDistance   = kmToMetres * (end - start)
        distancePerTurn = 2 * pi * r
        kmToMetres      = 1000

`where` clauses are useful for breaking down functions into simpler components, avoiding repetition and pattern matching.

In [2]:
normalise :: (Float, Float) -> (Float, Float)
-- In this case, sqrt is only evaluated once :)
normalise (x, y)
    = (x / m, y / m)
    where
        m = sqrt (x^2 + y^2)

In [ ]:
quotrem :: Int -> Int -> (Int, Int)
quotrem x y = (x `div` y, x `mod` y)

-- Converts distance in yards to miles, furloughs, yards
-- Pattern matching is useful here to avoid repetition
racelength :: Int -> (Int, Int, Int)
racelength y
    = (m, f, y'')
    where
        (m, y') = quotrem y 1760
        (f, y'') = quotrem y' 220

* You can define local functions too
    * Local functions cannot be anywhere else in the program
    * Order doesn't really matter (unless we're considering recursion, see next time :))

In [4]:
raceLength :: Int -> (Int, Int, Int)
raceLength y
    = (m, f, y'')
    where
        (m, y') = quotrem y 1760
        (f, y'') = quotrem y' 220
        
        -- Type signatures in local functions aren't necessary, but are recommended at this stage
        quotrem :: Int -> Int -> (Int, Int)
        quotrem x y = (x `div` y, x `mod` y)

* Type synonyms: naming types
    * Not defining a new type, just naming existing ones
    * Start with a capital letter, aid readability

In [8]:
type Mass = Float
type Position = (Float, Float)
type Force = (Float, Float)
type Object = (Mass, Position)

-- Look at how much simpler the type signature is now!
force :: Object -> Object -> Force
force (m1, (x1, y1)) (m2, (x2, y2))
    = (f * dx / r, f * dy / r)
    where 
        dx = abs (x1 - x2)
        dy = abs (y1 - y2)
        r = sqrt (dx^2 + dy^2)
        g = 6.67 * 10 ^ (-11)
        f = g * m1 * m2 / r^2

## Scope

The scope of an identifier is the part of the program in which the identifier has a meaning.
* Top-level identifiers: global scope
* Within each rule: each argument and local identifier is in scope throughout the rule
Identifiers in `where` clauses supersede argument identifiers with the same name.
* Identifiers in nested `where` clauses superseded those with the same name in an outer where clause
* tldr; think of it like subsets, if there are conflicts of variables choose the one most local (innermost)

In [11]:
f :: Int -> Int -> Int
f x y -- This y is superseded!
    = x + y -- Uses y from line 5, uses x from line 2
    where
        y = x^2 -- Uses x from line 6
            where x = 3
            
-- Hence the function has the same meaning as
-- f x y = x + 9

## Evaluation

* Evaluation: reducing expression to simplest normal form (cannot be simplified anymore)
    * E.g. 3 + 4 is reducible into the normal form 7
* Redex: reducible expression
    * Reduction: repeatedly reducing redexes until no more redexes exist

In [12]:
double :: Int -> Int
double x
    = x + x

* Call-by-value reduction sequence for `double (3 + 4)`
    * Evaluate 3 + 4: `double 7`, built-in rules for +
    * Evaluate `7 + 7`: output 14, built-in rules for `double`
* Call-by-name reduction sequence
    * Evaluate (3 + 4) + (4 + 4): rule for `double`
    * Evaluate 7 + &: built-in rules for +
    
Haskell favours call-by-name reduction!
* Stores as a graph: breaks it down into components
* Stores a single `x` value

Lazy evaluation: reduces a redex only if the value of the redex is required to produce the normal form
* Don't evaluate anything unless you have to

In [15]:
f :: Float -> Float -> Float
f x y
    | x < 0 = 0
    | otherwise = y

f (-3) (6/0) -- The second argument y is not needed

0.0